In [2]:
import nltk
import spacy
import pandas as pd
from tqdm import tqdm
import os
import nltk
import numpy as np
from pattern.text.en import singularize
import nltk
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
import pickle
from collections import Counter

In [1]:
1+2

3

# Getting sentence classifications

In [3]:
f = open("LitCovid_sent_classification_formatted.sents", "r")
sec_class_sents = f.read()
f = open("LitCovid_sent_classification_formatted.txt", "r")
sec_class_txt = f.read()

In [4]:
sec_pmids[6469]

NameError: name 'sec_pmids' is not defined

In [8]:
sec_classes = sec_class_sents.split("\n\n\n")
sec_pmids = [t.split(" | ")[0] for t in sec_class_txt.split("\n")]
sections = {}
for i in range(len(sec_pmids)):
    if sec_pmids[i] != "":
        sections[sec_pmids[i]] = [(t.split("||")[0], t.split("||")[1]) for t in sec_classes[i].split("\n")]

In [9]:
split_sections = {}
for i in tqdm(range(len(sec_pmids))):
    if sec_pmids[i] != "":
        sect = sections[sec_pmids[i]]
        temp_sect = []
        for line in sect:
            if len(line[1].split()) > 30:
                text = line[1].split()
#                 print(text)
                split_sentences = [' '.join(text[i:i+30]) for i in range(0, len(text),30)]
#                 print(split_sentences)
                for sent in split_sentences:
                    temp_sect.append((line[0], sent))
            else:
                temp_sect.append(line)
        
        split_sections[sec_pmids[i]] = temp_sect

100%|███████████████████████████████████████████████████████████████████████████| 6470/6470 [00:00<00:00, 12750.56it/s]


In [115]:
import re
t  = "They had a median ( interquartile range [ IQR ] ) age of 47.0 ( 38.0-70.3 ) years , and 16 ( 38.1 % ) were men .".split()
age = "".join(t[t.index("age")+1:t.index("years")])
regex = re.compile('[a-zA-Z]')
regex.sub('', age)

'47.0(38.0-70.3)'

In [132]:
list(re.finditer("age", "ages you know ages"))

[<re.Match object; span=(0, 3), match='age'>,
 <re.Match object; span=(14, 17), match='age'>]

In [179]:
# sec_classes[i].split("\n")
sections.get(dirs[3422].split(".")[0])

[('TITLE',
  'Potential effectiveness and safety of antiviral agents in children with coronavirus disease 2019: a rapid review and meta-analysis.'),
 ('BACKGROUND',
  'Background : The COVID - 19 outbreak presents a new , life - threatening disease .'),
 ('OBJECTIVE',
  'Our aim was to assess the potential effectiveness and safety of antiviral agents for COVID - 19 in children .'),
 ('METHODS',
  'Methods : Electronic databases ( MEDLINE , Embase , Web of Science , the Cochrane library , CBM , CNKI , and Wanfang Data ) from their inception to March 31 , 2020 were searched for randomized controlled trials ( RCTs ) , clinical controlled trials and cohort studies of interventions with antiviral agents for children ( less than 18 years of age ) with COVID - 19 .'),
 ('METHODS',
  'Results : A total of 23 studies with 6,008 patients were included .'),
 ('METHODS',
  'There was no direct evidence and all of evidence were indirect .'),
 ('METHODS',
  'The risks of bias in all studies were mod

In [201]:
custom_syns = [['COVID-19', "SARS-CoV-2", "SARS-CoV-2 infection", "COVID-19 infection", 
  "coronavirus disease 2019 ( COVID-19 )", 'Covid-19', 'laboratory-confirmed COVID-19', 
  "coronavirus disease 2019", "COVID-19 disease", "coronavirus disease", "SARS-CoV", 
  "COVID-19 positive", "SARS-COV-2", "SARS-CoV-2 infections",  "Covid-19 infection", 
  "COVID-19 infected", "COVID-19-positive", "SARS-CoV-2 infected", "SARS-CoV-2-infected",
  "2019-nCoV", "COVID-19 infections", "COVID 19", "COVID19", "Covid-19 infection",
  "severe acute respiratory syndrome coronavirus 2 ( SARS-CoV-2 ) infection", 
  'coronavirus disease ( COVID-19 )', "CoV infections", "coronavirus", "2019-new coronavirus",
  "SARS-CoV-2 virus", "coronavirus infections"],
 ["severe COVID-19", "severe and critical COVID-19", "severe COVID-19 infection", 
  'severe coronavirus disease 2019 ( COVID-19 )',
  'severe coronavirus disease 2019 ( COVID-19 )', 
  'severe acute respiratory syndrome ( SARS )', 'severe and critical COVID-19',
  'severe acute respiratory syndrome coronavirus 2',
  'critically ill COVID-19', "severe COVID-19 disease"],
 ["severe COVID-19 pneumonia", "severe pneumonia", "COVID-19 pneumonia", "SARS-CoV-2 pneumonia",
  'novel coronavirus pneumonia', "2019-nCoV pneumonia"],
 ["male", "men", "males", "male sex"],
 ["infection", "infected"],
 ["female", "women", "females", "female sex"],
 ["severe", "severe disease", "severe cases", "severely ill", "severe illness"],
 ["critical", "critically ill",  "critical illness"],
 ["comorbidity", "comorbidities"],
 ["ICU", "intensive care unit ( ICU )", "intensive care"],
 ["diabetes", "diabetes mellitus", "diabetic"],
 ["elderly", "older age", "older adults", "older"],
 ["hydroxychloroquine", "chloroquine", "hcq"],
 ["dead", "deceased", "mortality", "died", "death"]]

severe_subtags = [
    ["infection", "infections", "cases", "case", "outcomes", "outcome", "disease", "patient", "patients", "group"],
    ["pneumonia", "event", "events", "complications", "complication", "diabetes", "disease"],
    ["fever", "acute", "ards", "acidosis", "hypercoagulability", "inflammatory", "inflammation"],
    ["course", "courses", "stages", "stage"]]

severe_subtags_names = ["COVID-19", "other diseases", "symptoms", "course"]
age_subtags_names = ["median", "other diseases", "symptoms", "course"]
lower_syns = []
for syn in custom_syns:
    lower_syns.append([s.lower() for s in syn])

    
def get_syn(ent, orig_text, start):
    subtag = "None"
    covid_syns = ["covid", "sars-cov", "corona", "ncov", "cov infections"]
    severe_syns = ["severe"]
    if any(word in ent.lower() for word in covid_syns) and any(word in ent.lower() for word in severe_syns):
        ent = "severe covid-19"
    elif any(word in ent.lower() for word in covid_syns) and "pneumonia" in ent.lower():
        ent = "COVID-19 pneumonia"
#     elif any(word in ent.lower() for word in severe_syns):
#         return "severe"
    else:
        for l in lower_syns:
            if ent.lower() in l:
                ent =  l[0]
        
    if ent == "severe":
        for i in range(len(severe_subtags)):
            orig_text_split = orig_text.lower().split(" ")
            if len(orig_text_split) > start+1 and orig_text_split[start+1] in severe_subtags[i]:
                subtag = severe_subtags_names[i]
                
    if ent == "age":
        orig_text_split = orig_text.lower().split()
        orig_text_split = ["age" if text == "ages" else text for text in orig_text_split]
        
        if "child" in orig_text.lower():
            subtag = "<18"
        elif "years of age" in orig_text.lower():
            age_idx = orig_text_split.index("years")
            
            age = orig_text_split[age_idx-3:age_idx]
            if len(age) == 3:
                if age[1] == "under" or age[0] in ["younger", "less"]:
                    age = "<" + age[-1]

                elif age[1] == "over" or age[0] in ["older", "more"]:
                    age = ">" + age[-1]   

            subtag = "".join(age)
        
        elif "years" in orig_text.lower(): 
            age = orig_text_split[orig_text_split.index("age")+1:orig_text_split.index("years")]
            if "patients" in age and age[age.index("patients")-1].isnumeric():
                age = age[age.index("patients")+1:orig_text_split.index("years")]
                
            age = " ".join(age)
            age = age.replace("to", "-")
            regex = re.compile('[a-zA-Z]')
            subtag = regex.sub('', age)
        else:
            subtag = "concept"
        
        if "+-" in subtag:
            subtag = subtag.replace(" ", "")
            nums = [float(x) for x in subtag.split("+-")]
            subtag = nums[0]-nums[1]
        subtag = subtag.replace("[", "(")
        subtag = subtag.replace("]", ")")
    return ent, subtag

In [191]:
re.search("^[0-9><()]", "w123")

# Get data

In [202]:

folder = "../LitCovid_Treatment_json/"
dirs = os.listdir(folder)
pmids = []
entities = []

singulars = 0
for file in tqdm(dirs, desc = "dirs"):
# for file in dirs[:3]:
#     try:
    if file not in ["exceptionlist.txt", "label2id.pkl", "predict.tf_record"]:

        #get filepath and read in data
        file_path = os.path.join(folder, file)
        temp = pd.read_json(file_path)
        pmid = file[:-5]
        section = split_sections.get(pmid)

#             if section[0][0] == "TITLE":
#                 sec_idx = 1
#             else:
#                 sec_idx = 0
#         print(temp)
#         print(section)
        for i in range(temp.shape[0]):

            #get entities sentence by sentence
            ents_dict = temp.sentences.values[i].get("entities")
            orig_text = temp.sentences.values[i].get("text")

            if i < len(section):
                temp_sec = section[i][0]
#                     sec_idx +=1

            else:
                temp_sec = "ERROR"

            if len(ents_dict.keys()) > 0:
                temp_ents = [(ents_dict[k]["text"], ents_dict[k]["class"], ents_dict[k]["negation"], ents_dict[k]["start"], orig_text, "None", temp_sec, pmid) for k in ents_dict.keys()]
                true_ents = []
                for i in range(len(temp_ents)):
                    ent = temp_ents[i][0]
                    cl = temp_ents[i][1]
                    neg = temp_ents[i][2]
                    start = temp_ents[i][3]
                    subtag = temp_ents[i][5]
                    info_tuple = (new_ent, cl, neg, start, orig_text, subtag, temp_sec, pmid)
                    added = False

                    if len(ent) > 1: #make sure it's not punctuation

                        if len(ent.split()) > 1: #if phrase

                            #check to see if it's in custom set of synonyms
                            new_ent, subtag = get_syn(ent, orig_text, start)
                            info_tuple = (new_ent, cl, neg, start, orig_text, subtag, temp_sec, pmid)
                            true_ents.append(info_tuple)

                        else: #if single word
                            if ent.lower() not in stop_words: #if not stop word

                                #check to see if it's in custom set of synonyms
                                new_ent, subtag = get_syn(ent, orig_text, start)

                                if new_ent != ent:
                                    info_tuple = (new_ent, cl, neg, start, orig_text, subtag, temp_sec, pmid)
                                    true_ents.append(info_tuple)

                                else:
                                    ent = singularize(ent)
                                    new_ent, subtag = get_syn(ent, orig_text, start)
                                    info_tuple = (new_ent, cl, neg, start, orig_text, subtag, temp_sec, pmid)
                                    true_ents.append(info_tuple)       

                                    
                entities += true_ents
#     except:
#         print(pmid)
#         print(section)
#         print(temp.sentences.values)


dirs: 100%|███████████████████████████████████████████████████████████████████████| 6459/6459 [00:45<00:00, 141.82it/s]


In [203]:
ent_df = pd.DataFrame(entities, columns=["text", "type", "negation", "start", "sentence", "subtag", "section", "pmid"])
# ent_df.to_csv("LitCovid_treatment_ents.csv")
ent_df



,text,type,negation,start,sentence,subtag,section,pmid
0,covid-19,Participant,0,9,"In this review , we provide an update on CoV i...",None,BACKGROUND,31981224
1,relevant,Participant,0,12,"In this review , we provide an update on CoV i...",None,BACKGROUND,31981224
2,host defense against,Outcome,0,17,"In this review , we provide an update on CoV i...",None,BACKGROUND,31981224
3,covid-19,Participant,0,8,This highlights the importance of immune respo...,None,CONCLUSIONS,31981224
4,covid-19,Participant,0,14,"As of January 22 , 2020 , a total of 571 cases...",None,TITLE,31996494
5,inflammation,Outcome,0,9,"Here , we propose for the first time that infl...",None,RESULTS,32013309
6,covid-19,Participant,0,11,"Here , we propose for the first time that infl...",None,RESULTS,32013309
7,inhibited,modifier,0,13,"Here , we propose for the first time that infl...",None,RESULTS,32013309
8,accelerate,modifier,0,0,accelerate its control .,None,BACKGROUND,32015560
9,control,Outcome,0,2,accelerate its control .,None,BACKGROUND,32015560


In [169]:
print(ent_df[(ent_df.text == "age")].shape[0])
print(ent_df[(ent_df.text == "age") & (ent_df.subtag != "concept")].shape[0])

737
369


In [204]:
ent_df[(ent_df.text == "age")& (ent_df.subtag != "concept")]

,text,type,negation,start,sentence,subtag,section,pmid
147,age,Participant,0,12,A total of 14 children ( 6 months to 14 years ...,<18,RESULTS,32051073
719,age,Participant,0,2,The mean age of the 52 patients was 59 7 ( SD ...,59 7 ( 13 3 ),BACKGROUND,32105632
1728,age,Participant,0,8,"Results : Of 201 patients , the median age was...",51,BACKGROUND,32167524
1850,age,Participant,0,3,Results : The age of the 54 patients was 68.0 ...,"68.0 ( 59.8 , 74.3 )",METHODS,32171190
2123,age,Participant,0,2,The median age of 69 enrolled patients was 42....,69 42.0,RESULTS,32176772
2590,age,Participant,0,2,The median age was 47 years ( interquartile ra...,47,CONCLUSIONS,32198776
3111,age,Participant,0,5,The mean ( SD ) age was 58.50 ( 14.66 ) years .,58.50 ( 14.66 ),BACKGROUND,32219356
3238,age,Participant,0,6,"Results : All 5 patients ( age range , 36-65 y...",", 36-65",METHODS,32219428
3490,age,Participant,0,4,Results : The median age of 7 patients was 59 ...,59,METHODS,32220276
3552,age,Participant,0,22,Current research shows that the poor prognosis...,( > 60,RESULTS,32221983


In [67]:
ent_df[(ent_df.text == "severe") & (ent_df.subtag == "None")].sentence.values

array(['For cases of severe illness , early and also short periods of moderate glucocorticoid was supported .',
       'According to the relevant diagnostic criteria , the patients were divided into three groups : mild ( 15 cases ) , severe ( 9 cases ) and critical ( 5',
       'Severe or critically ill COVID-19 patients with myocardial injury face a significantly higher risk of in-hospital mortality .',
       'Compared to the mild cases , the severe ones had lower lymphocyte counts and higher plasma levels of Pt , APTT , d-dimer , lactate dehydrogenase , PCT , ALB',
       'Acute respiratory distress syndrome ( ARDS ) was the most severe pulmonary complication that caused high mortality rate .',
       'Among them , two common and one severe patient were recovered and discharged in 10 days after treatment .',
       'The symptoms in neonates were milder and outcomes were less severe compared to adults .',
       'subjects , 2 with light , common or severe subjects , 1 with light and 

In [140]:
file = "32013309.json"
file_path = os.path.join(folder, file)
temp = pd.read_json(file_path)
    
temp.sentences.values

array([{'Section': 'METHODS', 'text': 'Coronavirus can cause respiratory syndrome which to date has affected about twelve thousand individuals , especially in China .', 'entities': {}, 'relations': {}},
       {'Section': 'METHODS', 'text': 'Coronavirus is interspecies and can also be transmitted from man to man , with an incubation ranging from 1 to 14 days .', 'entities': {}, 'relations': {}},
       {'Section': 'METHODS', 'text': 'Human coronavirus infections can induce not only mild to severe respiratory diseases , but also inflammation , high fever , cough , acute respiratory tract infection and dysfunction of internal', 'entities': {}, 'relations': {}},
       {'Section': 'METHODS', 'text': 'organs that may lead to death .', 'entities': {}, 'relations': {}},
       {'Section': 'METHODS', 'text': 'Coronavirus infection ( regardless of the various types of corona virus ) is primarily attacked by immune cells including mast cells ( MCs ) , which are located in the', 'entities': {}, 

In [141]:
sections.get("32013309")

[('TITLE',
  'Mast cells contribute to coronavirus-induced inflammation: new anti-inflammatory strategy.'),
 ('BACKGROUND',
  'Coronavirus can cause respiratory syndrome which to date has affected about twelve thousand individuals , especially in China .'),
 ('RESULTS',
  'Coronavirus is interspecies and can also be transmitted from man to man , with an incubation ranging from 1 to 14 days .'),
 ('RESULTS',
  'Human coronavirus infections can induce not only mild to severe respiratory diseases , but also inflammation , high fever , cough , acute respiratory tract infection and dysfunction of internal organs that may lead to death .'),
 ('RESULTS',
  'Coronavirus infection ( regardless of the various types of corona virus ) is primarily attacked by immune cells including mast cells ( MCs ) , which are located in the submucosa of the respiratory tract and in the nasal cavity and represent a barrier of protection against microorganisms .'),
 ('RESULTS',
  'Viral activate MCs release early

# Send counters to pickle file

In [142]:
ent_df = pd.read_csv("LitCovid_treatment_ents.csv")
p_count = Counter(ent_df[ent_df.type == "Participant"].text.values)
i_count = Counter(ent_df[ent_df.type == "Intervention"].text.values)
o_count = Counter(ent_df[ent_df.type == "Outcome"].text.values)

counters = [p_count, i_count, o_count]
c_names = ["p_count", "i_count", "o_count"]
for i in range(len(counters)):
    with open("Pickles/" + c_names[i] + '.pickle', 'wb') as file:
        pickle.dump(counters[i], file)
    

# Building weighted graph

In [ ]:
from collections import Counter
import itertools as it

# get data and group
ent_df = pd.read_csv("LitCovid_treatment_ents.csv")
g = ent_df.groupby(["pmid", "type"])

#initialize variables
groups = list(g.groups)
types = ["Participant", "Outcome", "Intervention"]
edge_weight = Counter()
nodes = []

for p in tqdm(ent_df.pmid.values):
    temp = []
    for tup in list(it.product([p], types)):
        if tup in groups:
            
            #get pmid entities
            grp = g.get_group(tup)
            
            #format appropriately
            temp_nodes = ["{} ({})".format(t, tup[1][0]) for t in grp.text.values]
            temp.append(temp_nodes)
            nodes += temp_nodes
        else:
            temp.append([])
            
    #get edges
    products = [list(it.product(temp[0], temp[1])), list(it.product(temp[0], temp[2])), list(it.product(temp[1], temp[2]))]
    
    #add edges to adj list
    for prod in products:
        if prod != []:
            edge_weight.update(prod)

 28%|█████████████████▊                                              | 33980/121669 [03:07<12:23, 117.86it/s]

In [ ]:
with open("Pickles/" + 'wb') as file:
    pickle.dump(edge_weight, file)

In [ ]:
with open('edges.pickle', 'rb') as file:
    edge_weight = pickle.load(file)

In [ ]:
import networkx as nx
from pyvis.network import Network
import matplotlib.pyplot as plt

G = nx.Graph()
for item in edge_weight.most_common(200):
    G.add_edge(item[0][0], item[0][1], weight=item[1]*100)



g = Network(height=800, width=800, notebook=True)
g.barnes_hut()
g.from_nx(G)
g.show_buttons(filter_=['physics'])
g.show("ex.html")

